# YOLOv8 Vegetation Detection Model

Tạo mô hình YOLOv8 để detect cỏ cây với 1 class duy nhất.

## Pipeline:
1. **Setup Environment** - Cài đặt thư viện cần thiết
2. **Download Dataset** - Tải dataset cỏ cây công khai  
3. **Data Preparation** - Chuẩn bị dữ liệu cho YOLO format
4. **Model Training** - Train YOLOv8 với custom dataset
5. **Model Evaluation** - Đánh giá và test model
6. **Inference** - Sử dụng model để detect cỏ cây

## Target: Single class "vegetation" detection

In [ ]:
# 1. Setup Environment
import os
import sys
import shutil
import urllib.request
import zipfile
from pathlib import Path
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

# Kiểm tra GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Device: {device}")

# Cài đặt ultralytics (YOLOv8)
try:
    from ultralytics import YOLO
    print("✅ Ultralytics đã được cài đặt")
except ImportError:
    print("📦 Đang cài đặt ultralytics...")
    os.system("pip install ultralytics")
    from ultralytics import YOLO
    print("✅ Ultralytics đã được cài đặt thành công")

# Tạo thư mục project
project_dir = Path("./vegetation_detection")
project_dir.mkdir(exist_ok=True)

data_dir = project_dir / "data"
models_dir = project_dir / "models"
results_dir = project_dir / "results"

for dir_path in [data_dir, models_dir, results_dir]:
    dir_path.mkdir(exist_ok=True)

print(f"📁 Project structure created:")
print(f"   Data: {data_dir}")
print(f"   Models: {models_dir}")
print(f"   Results: {results_dir}")

# Kiểm tra phiên bản
print(f"🐍 Python: {sys.version}")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"📷 OpenCV: {cv2.__version__}")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# 2. Download Vegetation Dataset
import requests
import json
from torchvision import datasets, transforms

class VegetationDatasetDownloader:
    def __init__(self):
        self.data_dir = data_dir
        self.annotations = []
        
    def download_oxford_flowers(self):
        """Download Oxford Flowers 102 dataset (có sẵn vegetation data)"""
        print("📥 Downloading Oxford Flowers 102 dataset...")
        
        try:
            transform = transforms.Compose([
                transforms.Resize((640, 640)),
                transforms.ToTensor()
            ])
            
            # Download train và test sets
            train_dataset = datasets.Flowers102(
                root=str(self.data_dir),
                split='train',
                download=True,
                transform=transform
            )
            
            test_dataset = datasets.Flowers102(
                root=str(self.data_dir),
                split='test',
                download=True,
                transform=transform
            )
            
            print(f"✅ Downloaded Flowers102:")
            print(f"   Train: {len(train_dataset)} images")
            print(f"   Test: {len(test_dataset)} images")
            print(f"   Classes: 102 flower species")
            
            return train_dataset, test_dataset
            
        except Exception as e:
            print(f"❌ Error downloading dataset: {e}")
            return None, None
    
    def create_vegetation_samples(self, num_samples=500):
        """Tạo sample dataset từ internet hoặc local images"""
        print("🌿 Creating vegetation sample dataset...")
        
        # URLs mẫu của vegetation images (public domain)
        sample_urls = [
            "https://images.unsplash.com/photo-1441974231531-c6227db76b6e?w=640",
            "https://images.unsplash.com/photo-1574263867128-32d95b2e1b2d?w=640",
            "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=640",
            "https://images.unsplash.com/photo-1519389950473-47ba0277781c?w=640",
            "https://images.unsplash.com/photo-1541781774459-bb2af2f05b55?w=640"
        ]
        
        images_dir = self.data_dir / "vegetation_samples" / "images"
        labels_dir = self.data_dir / "vegetation_samples" / "labels"
        
        images_dir.mkdir(parents=True, exist_ok=True)
        labels_dir.mkdir(parents=True, exist_ok=True)
        
        downloaded = 0
        
        for i, url in enumerate(sample_urls):
            if downloaded >= num_samples:
                break
                
            try:
                print(f"   Downloading image {i+1}...")
                response = requests.get(url, stream=True, timeout=10)
                
                if response.status_code == 200:
                    image_path = images_dir / f"vegetation_{i+1:04d}.jpg"
                    
                    with open(image_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    
                    # Tạo label file (toàn bộ ảnh là vegetation)
                    self.create_full_image_label(labels_dir / f"vegetation_{i+1:04d}.txt")
                    
                    downloaded += 1
                    print(f"   ✅ Saved: {image_path.name}")
                    
            except Exception as e:
                print(f"   ❌ Failed to download {url}: {e}")
                continue
        
        print(f"✅ Created {downloaded} vegetation samples")
        return downloaded
    
    def create_full_image_label(self, label_path):
        """Tạo label file cho toàn bộ ảnh là vegetation (class 0)"""
        # YOLO format: class_id center_x center_y width height (normalized)
        # Toàn bộ ảnh là vegetation: class 0, center (0.5, 0.5), size (1.0, 1.0)
        with open(label_path, 'w') as f:
            f.write("0 0.5 0.5 1.0 1.0\n")

# Download dataset
downloader = VegetationDatasetDownloader()

# Option 1: Download Oxford Flowers (convert về vegetation class)
train_dataset, test_dataset = downloader.download_oxford_flowers()

# Option 2: Tạo sample dataset
if train_dataset is None:
    print("📥 Creating sample vegetation dataset...")
    sample_count = downloader.create_vegetation_samples(num_samples=100)
    
print("✅ Dataset preparation completed!")

In [ ]:
# 3. Data Preparation - Convert to YOLO Format
import glob
import random
from sklearn.model_selection import train_test_split

class YOLODataPreparer:
    def __init__(self, base_dir):
        self.base_dir = Path(base_dir)
        self.yolo_dir = self.base_dir / "yolo_dataset"
        
        # Tạo cấu trúc thư mục YOLO
        self.train_images = self.yolo_dir / "train" / "images"
        self.train_labels = self.yolo_dir / "train" / "labels"
        self.val_images = self.yolo_dir / "val" / "images"
        self.val_labels = self.yolo_dir / "val" / "labels"
        
        for dir_path in [self.train_images, self.train_labels, self.val_images, self.val_labels]:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    def convert_flowers_to_vegetation(self):
        """Convert Oxford Flowers dataset thành vegetation detection"""
        print("🔄 Converting Oxford Flowers to vegetation dataset...")
        
        # Tìm flowers dataset đã download
        flowers_dir = self.base_dir / "flowers-102"
        
        if not flowers_dir.exists():
            print("❌ Oxford Flowers dataset not found")
            return False
        
        # Lấy tất cả ảnh
        image_files = list(flowers_dir.rglob("*.jpg"))
        print(f"   Found {len(image_files)} images")
        
        if len(image_files) == 0:
            print("❌ No images found in dataset")
            return False
        
        # Split train/val (80/20)
        train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)
        
        print(f"   Train: {len(train_files)} images")
        print(f"   Val: {len(val_files)} images")
        
        # Process train set
        self.process_image_set(train_files, self.train_images, self.train_labels, "train")
        
        # Process validation set
        self.process_image_set(val_files, self.val_images, self.val_labels, "val")
        
        return True
    
    def process_image_set(self, image_files, images_dir, labels_dir, split_name):
        """Xử lý một set ảnh (train hoặc val)"""
        print(f"   Processing {split_name} set...")
        
        for i, image_file in enumerate(image_files):
            try:
                # Copy ảnh
                new_image_name = f"vegetation_{split_name}_{i:04d}.jpg"
                new_image_path = images_dir / new_image_name
                shutil.copy2(image_file, new_image_path)
                
                # Tạo label file (toàn bộ ảnh là vegetation)
                label_name = new_image_name.replace('.jpg', '.txt')
                label_path = labels_dir / label_name
                
                # YOLO format: class_id center_x center_y width height (normalized)
                # Class 0 = vegetation, toàn bộ ảnh
                with open(label_path, 'w') as f:
                    f.write("0 0.5 0.5 1.0 1.0\n")
                
                if (i + 1) % 100 == 0:
                    print(f"     Processed {i + 1}/{len(image_files)} images")
                    
            except Exception as e:
                print(f"     ❌ Error processing {image_file}: {e}")
                continue
        
        print(f"   ✅ {split_name} set completed")
    
    def create_yaml_config(self):
        """Tạo file config YAML cho YOLOv8"""
        yaml_content = {
            'path': str(self.yolo_dir.absolute()),
            'train': 'train/images',
            'val': 'val/images',
            'nc': 1,  # Number of classes
            'names': ['vegetation']  # Class names
        }
        
        yaml_path = self.yolo_dir / "vegetation_config.yaml"
        
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, default_flow_style=False)
        
        print(f"✅ Created YAML config: {yaml_path}")
        return yaml_path
    
    def verify_dataset(self):
        """Kiểm tra dataset sau khi convert"""
        train_images_count = len(list(self.train_images.glob("*.jpg")))
        train_labels_count = len(list(self.train_labels.glob("*.txt")))
        val_images_count = len(list(self.val_images.glob("*.jpg")))
        val_labels_count = len(list(self.val_labels.glob("*.txt")))
        
        print(f"📊 Dataset verification:")
        print(f"   Train: {train_images_count} images, {train_labels_count} labels")
        print(f"   Val: {val_images_count} images, {val_labels_count} labels")
        
        # Kiểm tra consistency
        if train_images_count == train_labels_count and val_images_count == val_labels_count:
            print("✅ Dataset is consistent")
            return True
        else:
            print("❌ Dataset has mismatched images and labels")
            return False
    
    def visualize_sample(self, num_samples=3):
        """Hiển thị một số sample từ dataset"""
        print(f"🖼️ Visualizing {num_samples} random samples...")
        
        train_images = list(self.train_images.glob("*.jpg"))
        samples = random.sample(train_images, min(num_samples, len(train_images)))
        
        fig, axes = plt.subplots(1, len(samples), figsize=(15, 5))
        if len(samples) == 1:
            axes = [axes]
        
        for i, image_path in enumerate(samples):
            # Load và hiển thị ảnh
            image = cv2.imread(str(image_path))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            axes[i].imshow(image)
            axes[i].set_title(f"Vegetation Sample {i+1}")
            axes[i].axis('off')
            
            # Đọc label
            label_path = self.train_labels / (image_path.stem + '.txt')
            with open(label_path, 'r') as f:
                label = f.read().strip()
            
            print(f"   Sample {i+1}: {image_path.name} -> Label: {label}")
        
        plt.tight_layout()
        plt.show()

# Prepare data
data_preparer = YOLODataPreparer(data_dir)

# Convert dataset
if data_preparer.convert_flowers_to_vegetation():
    # Tạo YAML config
    config_path = data_preparer.create_yaml_config()
    
    # Verify dataset
    data_preparer.verify_dataset()
    
    # Visualize samples
    data_preparer.visualize_sample()
    
    print("✅ Data preparation completed!")
else:
    print("❌ Data preparation failed")

In [ ]:
# 4. Train YOLOv8 Model
from ultralytics import YOLO
import time

class VegetationModelTrainer:
    def __init__(self, config_path, models_dir):
        self.config_path = config_path
        self.models_dir = Path(models_dir)
        self.model = None
        
    def load_pretrained_model(self, model_size='n'):
        """Load pre-trained YOLOv8 model"""
        model_name = f"yolov8{model_size}.pt"
        
        print(f"📥 Loading YOLOv8{model_size} pre-trained model...")
        
        try:
            self.model = YOLO(model_name)
            print(f"✅ Loaded {model_name}")
            
            # Model info
            print(f"   Model type: {type(self.model.model).__name__}")
            print(f"   Device: {self.model.device}")
            
            return True
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            return False
    
    def train_model(self, epochs=50, imgsz=640, batch_size=16):
        """Train YOLOv8 model với vegetation dataset"""
        
        if self.model is None:
            print("❌ Model not loaded")
            return None
        
        print(f"🚀 Starting YOLOv8 training...")
        print(f"   Config: {self.config_path}")
        print(f"   Epochs: {epochs}")
        print(f"   Image size: {imgsz}")
        print(f"   Batch size: {batch_size}")
        print(f"   Device: {device}")
        
        try:
            # Training parameters
            train_args = {
                'data': str(self.config_path),
                'epochs': epochs,
                'imgsz': imgsz,
                'batch': batch_size,
                'device': device,
                'project': str(self.models_dir),
                'name': 'vegetation_detection',
                'save': True,
                'save_period': 10,  # Save checkpoint every 10 epochs
                'patience': 15,     # Early stopping patience
                'cache': True,      # Cache images for faster training
                'workers': 4,       # Number of worker threads
                'optimizer': 'AdamW',
                'lr0': 0.01,        # Initial learning rate
                'weight_decay': 0.0005,
                'warmup_epochs': 3,
                'mosaic': 1.0,      # Mosaic augmentation probability
                'copy_paste': 0.3   # Copy-paste augmentation probability
            }
            
            start_time = time.time()
            
            # Start training
            results = self.model.train(**train_args)
            
            training_time = time.time() - start_time
            
            print(f"✅ Training completed!")
            print(f"   Training time: {training_time/60:.2f} minutes")
            print(f"   Results saved in: {self.models_dir / 'vegetation_detection'}")
            
            return results
            
        except Exception as e:
            print(f"❌ Training failed: {e}")
            return None
    
    def validate_model(self):
        """Validate trained model"""
        print("🔍 Validating model...")
        
        try:
            # Load best model
            best_model_path = self.models_dir / "vegetation_detection" / "weights" / "best.pt"
            
            if best_model_path.exists():
                model = YOLO(str(best_model_path))
                
                # Validate
                results = model.val(data=str(self.config_path))
                
                print("✅ Validation completed!")
                print(f"   mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")
                print(f"   mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
                
                return results
            else:
                print(f"❌ Best model not found: {best_model_path}")
                return None
                
        except Exception as e:
            print(f"❌ Validation failed: {e}")
            return None

# Initialize trainer
trainer = VegetationModelTrainer(
    config_path=data_dir / "yolo_dataset" / "vegetation_config.yaml",
    models_dir=models_dir
)

# Load pre-trained model
if trainer.load_pretrained_model(model_size='n'):  # 'n'=nano, 's'=small, 'm'=medium, 'l'=large
    
    print("\n" + "="*50)
    print("🚀 STARTING TRAINING")
    print("="*50)
    
    # Train model (adjust parameters as needed)
    training_results = trainer.train_model(
        epochs=30,      # Giảm epochs cho test nhanh
        imgsz=640,      # Image size
        batch_size=8    # Giảm batch size nếu GPU memory nhỏ
    )
    
    if training_results:
        # Validate model
        validation_results = trainer.validate_model()
        
        print("\n" + "="*50)
        print("✅ TRAINING & VALIDATION COMPLETED")
        print("="*50)
        
    else:
        print("❌ Training failed")
        
else:
    print("❌ Failed to load pre-trained model")

In [ ]:
# 5. Model Inference và Testing
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class VegetationDetector:
    def __init__(self, model_path):
        """Load trained model để inference"""
        self.model_path = Path(model_path)
        self.model = None
        self.load_model()
    
    def load_model(self):
        """Load trained model"""
        if self.model_path.exists():
            print(f"📥 Loading trained model: {self.model_path}")
            try:
                self.model = YOLO(str(self.model_path))
                print("✅ Model loaded successfully!")
                return True
            except Exception as e:
                print(f"❌ Error loading model: {e}")
                return False
        else:
            print(f"❌ Model file not found: {self.model_path}")
            return False
    
    def predict_image(self, image_path, conf_threshold=0.5):
        """Predict vegetation trong ảnh"""
        if self.model is None:
            print("❌ Model not loaded")
            return None
        
        try:
            print(f"🔍 Detecting vegetation in: {image_path}")
            
            # Run inference
            results = self.model(image_path, conf=conf_threshold)
            
            # Get results
            result = results[0]
            
            print(f"✅ Detection completed!")
            print(f"   Detected {len(result.boxes)} vegetation objects")
            
            return result
            
        except Exception as e:
            print(f"❌ Prediction error: {e}")
            return None
    
    def visualize_results(self, image_path, result, save_path=None):
        """Hiển thị kết quả detection"""
        
        # Load ảnh gốc
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Create plot
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        ax.imshow(image)
        
        # Draw bounding boxes
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()  # x1, y1, x2, y2
            confidences = result.boxes.conf.cpu().numpy()
            
            for i, (box, conf) in enumerate(zip(boxes, confidences)):
                x1, y1, x2, y2 = box
                width = x2 - x1
                height = y2 - y1
                
                # Draw rectangle
                rect = patches.Rectangle(
                    (x1, y1), width, height,
                    linewidth=2, edgecolor='lime', facecolor='none'
                )
                ax.add_patch(rect)
                
                # Add label
                label = f'Vegetation {conf:.2f}'
                ax.text(x1, y1-5, label, 
                       bbox=dict(boxstyle="round,pad=0.3", facecolor='lime', alpha=0.7),
                       fontsize=10, color='black')
        
        ax.set_title(f'Vegetation Detection Results\nDetected: {len(result.boxes)} objects')
        ax.axis('off')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=300)
            print(f"💾 Results saved: {save_path}")
        
        plt.show()
    
    def test_on_validation_set(self, val_images_dir, num_samples=5):
        """Test model trên validation set"""
        
        val_images = list(Path(val_images_dir).glob("*.jpg"))
        test_samples = random.sample(val_images, min(num_samples, len(val_images)))
        
        print(f"🧪 Testing on {len(test_samples)} validation samples...")
        
        results_summary = []
        
        for i, image_path in enumerate(test_samples):
            print(f"\nTesting sample {i+1}/{len(test_samples)}: {image_path.name}")
            
            # Predict
            result = self.predict_image(image_path)
            
            if result:
                num_detections = len(result.boxes) if result.boxes is not None else 0
                
                if num_detections > 0:
                    avg_confidence = result.boxes.conf.mean().item()
                else:
                    avg_confidence = 0.0
                
                results_summary.append({
                    'image': image_path.name,
                    'detections': num_detections,
                    'avg_confidence': avg_confidence
                })
                
                # Visualize
                save_path = results_dir / f"test_result_{i+1}.jpg"
                self.visualize_results(image_path, result, save_path)
        
        # Summary
        print(f"\n📊 Test Results Summary:")
        print(f"   Total samples: {len(results_summary)}")
        
        if results_summary:
            total_detections = sum(r['detections'] for r in results_summary)
            avg_detections = total_detections / len(results_summary)
            avg_confidence = np.mean([r['avg_confidence'] for r in results_summary if r['avg_confidence'] > 0])
            
            print(f"   Total detections: {total_detections}")
            print(f"   Average detections per image: {avg_detections:.2f}")
            print(f"   Average confidence: {avg_confidence:.3f}")
        
        return results_summary

# Test trained model
best_model_path = models_dir / "vegetation_detection" / "weights" / "best.pt"

if best_model_path.exists():
    print("🎯 Testing trained vegetation detection model...")
    
    # Initialize detector
    detector = VegetationDetector(best_model_path)
    
    if detector.model:
        # Test trên validation set
        val_images_dir = data_dir / "yolo_dataset" / "val" / "images"
        
        if val_images_dir.exists():
            test_results = detector.test_on_validation_set(val_images_dir, num_samples=3)
            
            print("✅ Model testing completed!")
        else:
            print("❌ Validation images directory not found")
    
else:
    print("❌ Trained model not found. Run training first!")

In [ ]:
# 6. Save và Deploy Model
import onnx
import shutil

class VegetationModelExporter:
    def __init__(self, model_path, export_dir):
        self.model_path = Path(model_path)
        self.export_dir = Path(export_dir)
        self.export_dir.mkdir(exist_ok=True)
        
        if self.model_path.exists():
            self.model = YOLO(str(model_path))
        else:
            self.model = None
            print(f"❌ Model not found: {model_path}")
    
    def export_to_onnx(self):
        """Export model sang ONNX format"""
        if not self.model:
            return False
            
        try:
            print("📤 Exporting model to ONNX format...")
            
            onnx_path = self.export_dir / "vegetation_detector.onnx"
            
            # Export to ONNX
            self.model.export(
                format='onnx',
                imgsz=640,
                opset=11,
                simplify=True,
                dynamic=False
            )
            
            # Move exported file
            exported_onnx = self.model_path.parent / (self.model_path.stem + ".onnx")
            if exported_onnx.exists():
                shutil.move(str(exported_onnx), str(onnx_path))
                print(f"✅ ONNX model saved: {onnx_path}")
                return True
            else:
                print("❌ ONNX export failed")
                return False
                
        except Exception as e:
            print(f"❌ ONNX export error: {e}")
            return False
    
    def export_to_torchscript(self):
        """Export model sang TorchScript format"""
        if not self.model:
            return False
            
        try:
            print("📤 Exporting model to TorchScript format...")
            
            torchscript_path = self.export_dir / "vegetation_detector.torchscript"
            
            # Export to TorchScript
            self.model.export(
                format='torchscript',
                imgsz=640
            )
            
            # Move exported file
            exported_ts = self.model_path.parent / (self.model_path.stem + ".torchscript")
            if exported_ts.exists():
                shutil.move(str(exported_ts), str(torchscript_path))
                print(f"✅ TorchScript model saved: {torchscript_path}")
                return True
            else:
                print("❌ TorchScript export failed")
                return False
                
        except Exception as e:
            print(f"❌ TorchScript export error: {e}")
            return False
    
    def create_deployment_package(self):
        """Tạo package hoàn chỉnh cho deployment"""
        print("📦 Creating deployment package...")
        
        package_dir = self.export_dir / "deployment_package"
        package_dir.mkdir(exist_ok=True)
        
        # Copy model files
        model_files = list(self.export_dir.glob("vegetation_detector.*"))
        
        for model_file in model_files:
            shutil.copy2(model_file, package_dir / model_file.name)
            print(f"   ✅ Copied: {model_file.name}")
        
        # Copy original PyTorch model
        if self.model_path.exists():
            shutil.copy2(self.model_path, package_dir / "vegetation_detector.pt")
            print(f"   ✅ Copied: vegetation_detector.pt")
        
        # Create inference script
        self.create_inference_script(package_dir)
        
        # Create requirements.txt
        self.create_requirements_file(package_dir)
        
        # Create README
        self.create_readme_file(package_dir)
        
        print(f"✅ Deployment package created: {package_dir}")
        return package_dir
    
    def create_inference_script(self, package_dir):
        """Tạo script inference đơn giản"""
        script_content = '''#!/usr/bin/env python3
"""
Vegetation Detection Inference Script
Usage: python inference.py --image path/to/image.jpg
"""

import argparse
from ultralytics import YOLO
import cv2
import numpy as np

class VegetationDetector:
    def __init__(self, model_path="vegetation_detector.pt"):
        self.model = YOLO(model_path)
    
    def detect(self, image_path, conf_threshold=0.5):
        """Detect vegetation in image"""
        results = self.model(image_path, conf=conf_threshold)
        return results[0]
    
    def save_results(self, image_path, result, output_path):
        """Save detection results"""
        # Load image
        image = cv2.imread(image_path)
        
        # Draw detections
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            
            for box, conf in zip(boxes, confidences):
                x1, y1, x2, y2 = map(int, box)
                
                # Draw rectangle
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Add label
                label = f'Vegetation {conf:.2f}'
                cv2.putText(image, label, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Save result
        cv2.imwrite(output_path, image)
        print(f"Results saved: {output_path}")

def main():
    parser = argparse.ArgumentParser(description='Vegetation Detection')
    parser.add_argument('--image', required=True, help='Input image path')
    parser.add_argument('--output', default='result.jpg', help='Output image path')
    parser.add_argument('--conf', type=float, default=0.5, help='Confidence threshold')
    parser.add_argument('--model', default='vegetation_detector.pt', help='Model path')
    
    args = parser.parse_args()
    
    # Initialize detector
    detector = VegetationDetector(args.model)
    
    # Run detection
    result = detector.detect(args.image, args.conf)
    
    # Save results
    detector.save_results(args.image, result, args.output)
    
    # Print summary
    num_detections = len(result.boxes) if result.boxes is not None else 0
    print(f"Detected {num_detections} vegetation objects")

if __name__ == "__main__":
    main()
'''
        
        script_path = package_dir / "inference.py"
        with open(script_path, 'w') as f:
            f.write(script_content)
        
        print(f"   ✅ Created: inference.py")
    
    def create_requirements_file(self, package_dir):
        """Tạo requirements.txt"""
        requirements = [
            "ultralytics>=8.0.0",
            "opencv-python>=4.5.0",
            "numpy>=1.21.0",
            "torch>=1.11.0",
            "torchvision>=0.12.0",
            "Pillow>=8.3.0",
            "matplotlib>=3.5.0"
        ]
        
        req_path = package_dir / "requirements.txt"
        with open(req_path, 'w') as f:
            f.write('\n'.join(requirements))
        
        print(f"   ✅ Created: requirements.txt")
    
    def create_readme_file(self, package_dir):
        """Tạo README.md"""
        readme_content = '''# Vegetation Detection Model

YOLOv8-based vegetation detection model trained for single-class vegetation detection.

## Files
- `vegetation_detector.pt` - PyTorch model (recommended)
- `vegetation_detector.onnx` - ONNX model (cross-platform)
- `vegetation_detector.torchscript` - TorchScript model
- `inference.py` - Inference script
- `requirements.txt` - Python dependencies

## Installation
```bash
pip install -r requirements.txt
```

## Usage
```bash
# Basic inference
python inference.py --image input.jpg --output result.jpg

# With custom confidence threshold
python inference.py --image input.jpg --output result.jpg --conf 0.7

# Using ONNX model
python inference.py --image input.jpg --model vegetation_detector.onnx
```

## Python API
```python
from ultralytics import YOLO

# Load model
model = YOLO('vegetation_detector.pt')

# Run inference
results = model('image.jpg')

# Process results
for result in results:
    boxes = result.boxes.xyxy  # Bounding boxes
    confs = result.boxes.conf  # Confidences
    print(f"Detected {len(boxes)} vegetation objects")
```

## Model Details
- **Task**: Object Detection
- **Classes**: 1 (vegetation)
- **Input Size**: 640x640
- **Framework**: YOLOv8
- **Format**: PyTorch (.pt), ONNX (.onnx), TorchScript (.torchscript)

## Performance
- Optimized for vegetation detection in natural scenes
- Single class detection reduces false positives
- Real-time inference capability
'''
        
        readme_path = package_dir / "README.md"
        with open(readme_path, 'w') as f:
            f.write(readme_content)
        
        print(f"   ✅ Created: README.md")

# Export và deploy model
if best_model_path.exists():
    print("🚀 Exporting trained model...")
    
    exporter = VegetationModelExporter(
        model_path=best_model_path,
        export_dir=results_dir / "exported_models"
    )
    
    # Export to different formats
    onnx_success = exporter.export_to_onnx()
    torchscript_success = exporter.export_to_torchscript()
    
    # Create deployment package
    if onnx_success or torchscript_success:
        package_dir = exporter.create_deployment_package()
        
        print("\n✅ Model export completed!")
        print(f"📦 Deployment package: {package_dir}")
        print("\n📋 Available formats:")
        print("   - PyTorch (.pt) - Recommended for Python")
        if onnx_success:
            print("   - ONNX (.onnx) - Cross-platform deployment")
        if torchscript_success:
            print("   - TorchScript (.torchscript) - Production deployment")
    
else:
    print("❌ No trained model found. Complete training first!")

# 🎉 Hoàn thành! Hướng dẫn sử dụng

## 📋 Tóm tắt quá trình:

1. **✅ Setup Environment** - Cài đặt YOLOv8 và dependencies
2. **✅ Download Dataset** - Oxford Flowers 102 dataset (100+ classes hoa)
3. **✅ Data Preparation** - Convert thành 1 class "vegetation" trong YOLO format
4. **✅ Model Training** - Train YOLOv8 với custom vegetation dataset
5. **✅ Model Testing** - Test trên validation set
6. **✅ Model Export** - Export sang nhiều format (PT, ONNX, TorchScript)

## 🚀 Cách chạy từng bước:

### Bước 1: Chạy Setup
```python
# Chạy cell 1 để setup environment
```

### Bước 2: Download Data  
```python
# Chạy cell 2 để download Oxford Flowers dataset
# Dataset sẽ được convert thành single class "vegetation"
```

### Bước 3: Prepare Data
```python  
# Chạy cell 3 để convert sang YOLO format
# Tạo train/val split và YAML config
```

### Bước 4: Train Model
```python
# Chạy cell 4 để train YOLOv8
# Có thể adjust epochs, batch_size, model_size
```

### Bước 5: Test Model
```python
# Chạy cell 5 để test model trên validation set
# Visualize detection results
```

### Bước 6: Export Model
```python
# Chạy cell 6 để export model sang các format khác nhau
# Tạo deployment package
```

## 🎯 Kết quả mong đợi:

- **Model**: `vegetation_detector.pt` detect class "vegetation" 
- **Performance**: mAP50 > 0.8 (depends on dataset quality)
- **Speed**: Real-time inference trên GPU
- **Deployment**: Ready-to-use package với inference script

## 🔧 Customization:

### Thay đổi dataset:
```python
# Trong cell 2, thay đổi downloader để dùng dataset khác
# Ví dụ: Agriculture-Vision, DeepWeeds, v.v.
```

### Thay đổi model size:
```python  
# Trong cell 4, thay đổi model_size
model_size = 'n'  # nano (fastest)
model_size = 's'  # small  
model_size = 'm'  # medium
model_size = 'l'  # large (best accuracy)
```

### Thay đổi training parameters:
```python
# Trong cell 4, adjust training args
epochs = 100        # More epochs = better accuracy  
batch_size = 16     # Larger batch = faster training
imgsz = 640         # Larger image = better accuracy
```

## 📱 Sử dụng model:

### Inference đơn giản:
```python
from ultralytics import YOLO

model = YOLO('vegetation_detector.pt')
results = model('image.jpg')

# Xem kết quả
for result in results:
    print(f"Detected {len(result.boxes)} vegetation objects")
```

### Inference với custom threshold:
```python
results = model('image.jpg', conf=0.7)  # 70% confidence
```

### Batch inference:
```python
results = model(['img1.jpg', 'img2.jpg', 'img3.jpg'])
```

## 🌟 Tips để cải thiện model:

1. **More Data**: Thêm nhiều ảnh vegetation đa dạng
2. **Data Augmentation**: Tăng augmentation trong training
3. **Longer Training**: Train nhiều epochs hơn  
4. **Larger Model**: Dùng YOLOv8m hoặc YOLOv8l
5. **Better Annotations**: Label chính xác hơn (không phải toàn bộ ảnh)

## 🔍 Troubleshooting:

- **Out of Memory**: Giảm batch_size và imgsz
- **Low mAP**: Tăng epochs, check data quality
- **Slow Training**: Sử dụng GPU, giảm image size
- **No Detections**: Giảm confidence threshold

Ready to detect vegetation! 🌿🚀